# Data Preparation

The source data was provided as Excel sheets by the Inter-American Dialogue, a US-based think tank focused on fostering democratic governance, social equity, and economic prosperity in the Americas through open dialogue and collaboration. This data was used in 2021 to generate a report and [database](https://www.thedialogue.org/amazon-energy-and-mining-database/). The Amazon Mining and Energy database is a resource that compiles information on mining and energy projects in the Amazon region, aiming to provide insights into their environmental, social, and economic impacts. 

In [16]:
# Load libraries
import os
import string
import unidecode
import random
import json

import pandas as pd
import geopandas as gpd
import plotly.express as px

import warnings
from IPython.core.interactiveshell import InteractiveShell

In [2]:
# Notebook configurations
warnings.filterwarnings('ignore')
pd.set_option('display.max.columns', None)
InteractiveShell.ast_node_interactivity = 'all'

In [3]:
# Functions for use across notebook

def generate_random_id(length, id_col):
    characters = string.ascii_letters.lower() + string.digits 
    original_id = False

    while original_id is False:
        new_id = ''.join(random.choice(characters) for _ in range(length))
        if new_id not in id_col:
            original_id = True
            break
    
    return new_id

# Format string data to follow a uniform format
def normalize_text_list(text_list: list, to_replace: str = "", replace_with: str = "", remove: list or str = ""):
    return list(
        map(
            lambda x: unidecode.unidecode(str(x).translate(str(x).maketrans(to_replace, replace_with, remove))).lower().strip().strip("_"), text_list
        )
    )

# Save dataframe as a CSV
def save_df_as_csv(df, file_path):
    try:
        os.makedirs(file_path.rsplit("/", 1)[0], exist_ok=True)
        df.to_csv(file_path, index=False)
    except:
        raise


## I. Convert source files to Pandas DataFrames

In [4]:
# Load the original source files
project_path = "../data/source/consolidated_amazon_projects_data.xlsx"
project_coords = "../data/source/project_coordinates_amazon_energy_and_mining.xlsx"

# Load a JSON mapping each subsidiary name to the company they belong to
owner_map_json = '../data/reference/owner_map.json'


In [5]:
mining_df = pd.read_excel(project_path, sheet_name="Mining")
oil_df = pd.read_excel(project_path, sheet_name="Oil & Gas")
hydro_df = pd.read_excel(project_path, sheet_name="Hydropower")

In [6]:
# Get the column names (headers) as sets for each DataFrame
hydro_columns = set(hydro_df.columns)
mining_columns = set(mining_df.columns)
oil_columns = set(oil_df.columns)

# Find the differences between the column sets
different_columns = {
    'hydro': hydro_columns - (mining_columns | oil_columns),
    'mining': mining_columns - (hydro_columns | oil_columns),
    'oil': oil_columns - (hydro_columns | mining_columns)
}

# Display the different column names for each DataFrame
for df_name, diff_columns in different_columns.items():
    if diff_columns:
        print(f"Different columns in {df_name}_df: {diff_columns}")
    else:
        print(f"No different columns in {df_name}_df")


Different columns in hydro_df: {'Installed Capacity (MW)'}
Different columns in mining_df: {'Substance(s)', 'Deal Type'}
No different columns in oil_df


__Note:__ Important to take into consideration the columns that are unique to each product type.

In [7]:
# Concatenate the DataFrames into a single DataFrame
df = pd.concat([hydro_df, oil_df, mining_df], ignore_index=True)

## II. Clean and correct mismatched data

### a. Format columns

In [8]:
# Normalize column headers 
df.columns = normalize_text_list(df.columns.values, " ", "_", ''.join([s for s in string.punctuation if s != "_"]))

# Normalize all text data
norm_cols = [col for (col, dtype) in df.dtypes.items() if dtype == "object"]

In [9]:
# Fix issue with duplicate 'deal_type' columns

df['deal_structure'] = df['deal_type'].iloc[:, 0] 
df['deal_mining'] = df['deal_type'].iloc[:, 1]

df = df.drop(columns=['deal_type'])
df.head()

,project_name,sector_type,country,installed_capacity_mw,operational_status,owner,equity_stake_owner,equity_stake_operator,country_of_company_hq,company_type,company_type_for_label,is_operator,lender_type,substances,deal_structure,deal_mining
0,Ambrosia,Hydropower,Bolivia,84.9,Permitting,Empresa Eléctrica Corani,1.0,1.0,Bolivia,SOE,State-owned Enterprise,Yes,Multilateral Development Bank,NaN,Greenfield,NaN
1,Banda Azul,Hydropower,Bolivia,146.0,Permitting,Empresa Eléctrica Corani,1.0,1.0,Bolivia,SOE,State-owned Enterprise,Yes,Foreign Development Bank/Fund,NaN,Greenfield,NaN
2,Chojlla,Hydropower,Bolivia,37.0,In Operation,Hidroeléctrica Boliviana,1.0,1.0,Bolivia,Local Power Company,Local Power Company,Yes,NaN,NaN,Greenfield,NaN
3,Corani,Hydropower,Bolivia,54.0,In Operation,Empresa Eléctrica Corani,1.0,1.0,Bolivia,SOE,State-owned Enterprise,Yes,Multilateral Development Bank,NaN,Greenfield,NaN
4,Ivirizu,Hydropower,Bolivia,279.8,Under Construction,Ende Valle Hermoso,1.0,0.0,Bolivia,SOE,State-owned Enterprise,No,NaN,NaN,Greenfield,NaN


In [10]:
# Create a new column 'deal_difference' to indicate differences
df['deal_difference'] = df['deal_structure'] != df['deal_mining']

# Show which sectors have differences
set(df[df['deal_difference'] == True]['sector_type'])


{'Hydropower', 'Mining', 'Oil & Gas'}

In [11]:
# Create a mask for rows where 'sector_type' equals 'mining'
mining_mask = df['sector_type'] == 'Mining'

# Use the mask to update the 'deal_structure' column
df.loc[mining_mask, 'deal_structure'] = df.loc[mining_mask, 'deal_mining']

df.tail()

,project_name,sector_type,country,installed_capacity_mw,operational_status,owner,equity_stake_owner,equity_stake_operator,country_of_company_hq,company_type,company_type_for_label,is_operator,lender_type,substances,deal_structure,deal_mining,deal_difference
549,Vermelho,Mining,Brazil,NaN,NaN,Horizonte Minerals,1.0,1.0,UK,Junior Exploration,Junior Exploration,Yes,NaN,"Cobalt, Nickel",Full Asset Transaction,Full Asset Transaction,True
550,Veta Islay,Mining,Peru,NaN,NaN,Compañía Minera Volcan,1.0,1.0,Peru,Junior Exploration,Junior Exploration,Yes,NaN,"Copper, Lead, Silver, Zinc",Greenfield,Greenfield,True
551,Volta Grande,Mining,Brazil,NaN,NaN,Belo Sun Mining,1.0,1.0,Canada,Small to Mid-tier Producer,Small to Mid-tier Producer,Yes,"Investment Bank, Commercial Bank, Holding Comp...",Gold,Greenfield,Greenfield,True
552,Warintza,Mining,Ecuador,NaN,NaN,Solaris Resources,1.0,1.0,Canada,Junior Exploration,Junior Exploration,Yes,NaN,"Copper, Molybdenum",Merger/Acquisition,Merger/Acquisition,True
553,Wynamu,Mining,Guyana,NaN,NaN,Guyana Goldfields,1.0,1.0,China,SOE,State-owned Enterprise,Yes,NaN,Gold,Greenfield,Greenfield,True


In [12]:
# Drop the duplicate column and change name of deal_structure back to deal_type
df = df.drop(columns=["deal_mining","deal_difference"])
df.rename(columns={"deal_structure":"deal_type"}, inplace=True)

df.tail()

,project_name,sector_type,country,installed_capacity_mw,operational_status,owner,equity_stake_owner,equity_stake_operator,country_of_company_hq,company_type,company_type_for_label,is_operator,lender_type,substances,deal_type
549,Vermelho,Mining,Brazil,NaN,NaN,Horizonte Minerals,1.0,1.0,UK,Junior Exploration,Junior Exploration,Yes,NaN,"Cobalt, Nickel",Full Asset Transaction
550,Veta Islay,Mining,Peru,NaN,NaN,Compañía Minera Volcan,1.0,1.0,Peru,Junior Exploration,Junior Exploration,Yes,NaN,"Copper, Lead, Silver, Zinc",Greenfield
551,Volta Grande,Mining,Brazil,NaN,NaN,Belo Sun Mining,1.0,1.0,Canada,Small to Mid-tier Producer,Small to Mid-tier Producer,Yes,"Investment Bank, Commercial Bank, Holding Comp...",Gold,Greenfield
552,Warintza,Mining,Ecuador,NaN,NaN,Solaris Resources,1.0,1.0,Canada,Junior Exploration,Junior Exploration,Yes,NaN,"Copper, Molybdenum",Merger/Acquisition
553,Wynamu,Mining,Guyana,NaN,NaN,Guyana Goldfields,1.0,1.0,China,SOE,State-owned Enterprise,Yes,NaN,Gold,Greenfield


In [13]:
# Normalize text data in dataframe
for col in norm_cols:
    df[col] = normalize_text_list(df[col], " ", "_", ''.join([s for s in string.punctuation if s != "_"]))

df.head()

,project_name,sector_type,country,installed_capacity_mw,operational_status,owner,equity_stake_owner,equity_stake_operator,country_of_company_hq,company_type,company_type_for_label,is_operator,lender_type,substances,deal_type
0,ambrosia,hydropower,bolivia,84.9,permitting,empresa_electrica_corani,1.0,1.0,bolivia,soe,stateowned_enterprise,yes,multilateral_development_bank,nan,greenfield
1,banda_azul,hydropower,bolivia,146.0,permitting,empresa_electrica_corani,1.0,1.0,bolivia,soe,stateowned_enterprise,yes,foreign_development_bankfund,nan,greenfield
2,chojlla,hydropower,bolivia,37.0,in_operation,hidroelectrica_boliviana,1.0,1.0,bolivia,local_power_company,local_power_company,yes,nan,nan,greenfield
3,corani,hydropower,bolivia,54.0,in_operation,empresa_electrica_corani,1.0,1.0,bolivia,soe,stateowned_enterprise,yes,multilateral_development_bank,nan,greenfield
4,ivirizu,hydropower,bolivia,279.8,under_construction,ende_valle_hermoso,1.0,0.0,bolivia,soe,stateowned_enterprise,no,nan,nan,greenfield


In [14]:
print(df.info(show_counts=True))
print("===========Null Values=============="*3)
print(df.isna().sum())
print("============Unique Values============="*3)
print(df.nunique())
print("========================="*3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 554 entries, 0 to 553
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   project_name            554 non-null    object 
 1   sector_type             554 non-null    object 
 2   country                 554 non-null    object 
 3   installed_capacity_mw   100 non-null    float64
 4   operational_status      554 non-null    object 
 5   owner                   554 non-null    object 
 6   equity_stake_owner      554 non-null    float64
 7   equity_stake_operator   554 non-null    float64
 8   country_of_company_hq   554 non-null    object 
 9   company_type            554 non-null    object 
 10  company_type_for_label  554 non-null    object 
 11  is_operator             554 non-null    object 
 12  lender_type             554 non-null    object 
 13  substances              554 non-null    object 
 14  deal_type               554 non-null    ob

### b. Correct company names

This had to be a manual effort, checking each company and doing some research online to identify which companies operate under different names or use multiple subsidiaries.

In [18]:
# Open the JSON file with company mapping
with open(owner_map_json, 'r') as jf:
    owner_mapping_dict = json.load(jf)

# Check the structure
print(json.dumps(owner_mapping_dict, indent=4))

{
    "amazonia_energia": [
        "amazonia_energia_sa",
        "caixa_fundo_de_investimentos_em_participacoes_amazonia_energia"
    ],
    "goldmining": [
        "gold_x_mining_corp"
    ],
    "perenco": [
        "perenco_peru_petroleum",
        "perenco_venezuela_petroleo_y_gas"
    ],
    "orion": [
        "orionoil_er_sa",
        "orion_energy_ocanopb"
    ],
    "amerisur": [
        "amerisur_resources",
        "amerisur_exploracion_colombia"
    ],
    "gran_tierra_energy": [
        "gran_tierra_energy_colombia"
    ],
    "repsol": [
        "repsol_ecuador",
        "repsol_ep_bolivia",
        "repsol_exploracion_guyana"
    ],
    "odebrecht": [
        "odebrecht_energia_do_brasil",
        "odebrecht_energia_sa"
    ],
    "engie": [
        "engie_energia_peru"
    ],
    "rio_tinto": [
        "rio_tinto_minera_peru_limitada"
    ],
    "geopark": [
        "geopark_colombia",
        "geopark_peru"
    ],
    "staatsolie": [
        "staatsolie_maatschappij_s

In [19]:
valid_list = []

# Loop through the values in the dictionary and extend the combined_list
for val in owner_mapping_dict.values():
    valid_list.extend(val)

In [20]:
# Extract unique values from 'owner' column that are not in valid_list
unique_owners = df['owner'].unique()
filtered_owners = [owner for owner in unique_owners if owner not in valid_list]

# Sort the filtered owners alphabetically
filtered_owners.sort()

# Print the filtered and sorted owners
for owner in filtered_owners:
    print(owner)

aguaytia_energy_del_peru
alcoa_corporation
alicanto_minerals
alta_floresta_gold_mineracao
altamira_gold
alupar_investimento
amazonas_geracao_e_transmissao_de_energia
amodaimi_oil_co
andes_petroleum_ecuador
anglo_american
apoio_engenharia_e_mineracao
ariana_operaciones_mineras
arrow_exploration_corp
auplata_mining_group
aura_minerals
aurania_resources
avanco_resources
barrick_gold
bbx_minerals
belo_sun_mining
bhp_group
boa_vista_gold
brazauro_recursos_minerais
cabral_gold
campo_puma_oriente
canacol_energy
canadian_energy_enterprises_cee_srl
capstone_mining
cementos_selva
cemig_geracao_e_transmissao
centaurus_metals
central_hidroelectrica_huallaga_hydro
centromin_peru
cerro_del_aguila_sa
china_gezhouba_group
china_national_petroleum_corporation_cnpc
china_three_gorges_corporation
chinango_sac
cinco_estrelas_agropecuaria_e_participacoes
citic_metal_co_ltd
codelco_do_brasil_mineracao
colossus_minerals
columbus_energy_resources
compagnie_miniere_esperance
companhia_brasileira_de_aluminio
co

### c. Add new columns

All duplicates were accounted for; two columns `parent_org` and `pid` (project ID) are created to show the hierarchy between projects and have an easier way of interacting with them.

In [21]:
parent_list = []

for row, data in df.iterrows():
    #print(row, data['owner'])
    if data['owner'] in owner_mapping_dict.keys():
            parent_list.append((row, data['owner']))
    elif data['owner'] in valid_list:
        for key, val in owner_mapping_dict.items():
            if data['owner'] in val:
                parent_list.append((row, key))
    else:
        parent_list.append((row, data['owner'] ))

print(parent_list)
            

[(0, 'ende'), (1, 'ende'), (2, 'hidroelectrica_boliviana'), (3, 'ende'), (4, 'ende'), (5, 'ende'), (6, 'ende'), (7, 'ende'), (8, 'ende'), (9, 'ende'), (10, 'hidroelectrica_boliviana'), (11, 'amazonas_geracao_e_transmissao_de_energia'), (12, 'odebrecht'), (13, 'amazonia_energia'), (14, 'eletrobras'), (15, 'entidades_de_previdencia_complementar'), (16, 'jmalucelli_energia'), (17, 'vale'), (18, 'companhia_paranaense_de_energia_sa'), (19, 'eletrobras'), (20, 'eletrobras'), (21, 'companhia_hidroeletrica_do_sao_francisco'), (22, 'mineracao_santa_elina_industria_e_comercio'), (23, 'cinco_estrelas_agropecuaria_e_participacoes'), (24, 'queiroz_galvao_energetica'), (25, 'companhia_hidroeletrica_do_sao_francisco'), (26, 'eletrosul'), (27, 'itamarati_norte'), (28, 'eletrobras'), (29, 'amazonia_energia'), (30, 'cemig_geracao_e_transmissao'), (31, 'eletrobras'), (32, 'odebrecht'), (33, 'saag_investimentos'), (34, 'eletrobras'), (35, 'eletrosul'), (36, 'eletrobras'), (37, 'odebrecht'), (38, 'eletrobr

In [22]:
# Create parent column
df['parent_org'] = [v for (k,v) in parent_list]

# Generate a column of random UUIDs
df['pid'] = [0] * len(df)
df['pid'] = [generate_random_id(6, df['pid']) for _ in range(len(df))]

df.head(10)

,project_name,sector_type,country,installed_capacity_mw,operational_status,owner,equity_stake_owner,equity_stake_operator,country_of_company_hq,company_type,company_type_for_label,is_operator,lender_type,substances,deal_type,parent_org,pid
0,ambrosia,hydropower,bolivia,84.9,permitting,empresa_electrica_corani,1.0,1.0,bolivia,soe,stateowned_enterprise,yes,multilateral_development_bank,nan,greenfield,ende,yrqfe4
1,banda_azul,hydropower,bolivia,146.0,permitting,empresa_electrica_corani,1.0,1.0,bolivia,soe,stateowned_enterprise,yes,foreign_development_bankfund,nan,greenfield,ende,zcera8
2,chojlla,hydropower,bolivia,37.0,in_operation,hidroelectrica_boliviana,1.0,1.0,bolivia,local_power_company,local_power_company,yes,nan,nan,greenfield,hidroelectrica_boliviana,q72ypq
3,corani,hydropower,bolivia,54.0,in_operation,empresa_electrica_corani,1.0,1.0,bolivia,soe,stateowned_enterprise,yes,multilateral_development_bank,nan,greenfield,ende,j5culn
4,ivirizu,hydropower,bolivia,279.8,under_construction,ende_valle_hermoso,1.0,0.0,bolivia,soe,stateowned_enterprise,no,nan,nan,greenfield,ende,f4rh3c
5,misicuni,hydropower,bolivia,185.5,in_operation,empresa_nacional_de_electricidad_ende,0.5,1.0,bolivia,soe,stateowned_enterprise,yes,multilateral_development_bank_multilateral_dev...,nan,greenfield,ende,tdzmhx
6,palillada,hydropower,bolivia,118.0,under_construction,empresa_electrica_corani,1.0,1.0,bolivia,soe,stateowned_enterprise,yes,nan,nan,greenfield,ende,sqqjkd
7,san_jose_cora,hydropower,bolivia,124.0,in_operation,empresa_electrica_corani,1.0,1.0,bolivia,soe,stateowned_enterprise,yes,multilateral_development_bank,nan,greenfield,ende,kd3nhv
8,santa_isabel_corani,hydropower,bolivia,93.4,in_operation,empresa_electrica_corani,1.0,1.0,bolivia,soe,stateowned_enterprise,yes,multilateral_development_bank,nan,greenfield,ende,bxz20c
9,umapalca,hydropower,bolivia,85.0,under_construction,empresa_electrica_corani,1.0,1.0,bolivia,soe,stateowned_enterprise,yes,nan,nan,greenfield,ende,it80of


In [23]:
# Fix sectory_type value for oil and gas
df['sector_type'] = df['sector_type'].str.replace('oil__gas', 'oil_gas')

## III. Geospatial Data

In [24]:
# Load coordinates dataset
hydro_coords = pd.read_excel(project_coords, sheet_name="Hydro")
oil_coords = pd.read_excel(project_coords, sheet_name="Oil & Gas")
mining_coords = pd.read_excel(project_coords, sheet_name="Mining")

print(hydro_df.shape, hydro_coords.shape)
print(oil_df.shape, oil_coords.shape)
print(mining_df.shape, mining_coords.shape)

(100, 14) (100, 3)
(302, 12) (302, 3)
(152, 13) (152, 3)


In [25]:
# Add sector_type column 
hydro_coords['sector_type'] = "hydropower"
oil_coords['sector_type'] = "oil_gas"
mining_coords['sector_type'] = "mining"

# Join the datasets
coords = pd.concat([hydro_coords, oil_coords, mining_coords], ignore_index=True)

In [26]:
# Normalize column headers
coords.columns = normalize_text_list(coords.columns.values, " ", "_", ''.join([s for s in string.punctuation if s != "_"]))

# Normalize values in project_name column
coords['project_name'] = normalize_text_list(coords['project_name'], " ", "_", ''.join([s for s in string.punctuation if s != "_"]))

coords.head()

,project_name,latitude,longitude,sector_type
0,abanico,-2.25617,-78.19793,hydropower
1,afobaka_brokopondo,5.00000,-55.00000,hydropower
2,ambrosia,-17.00910,-65.64090,hydropower
3,angel_i_ii_iii,-13.66330,-70.53820,hydropower
4,anto_ruiz,-13.88330,-70.20000,hydropower


In [27]:
# Create geometric points for each coordinate using GeoPandas
geo_coords = gpd.GeoDataFrame(coords, geometry=gpd.points_from_xy(coords['longitude'],coords['latitude']))
geo_coords.crs = {'init':'epsg:4326'}

In [28]:
geo_coords.head()

,project_name,latitude,longitude,sector_type,geometry
0,abanico,-2.25617,-78.19793,hydropower,POINT (-78.19793 -2.25617)
1,afobaka_brokopondo,5.00000,-55.00000,hydropower,POINT (-55.00000 5.00000)
2,ambrosia,-17.00910,-65.64090,hydropower,POINT (-65.64090 -17.00910)
3,angel_i_ii_iii,-13.66330,-70.53820,hydropower,POINT (-70.53820 -13.66330)
4,anto_ruiz,-13.88330,-70.20000,hydropower,POINT (-70.20000 -13.88330)


## IV. Save data as .CSV 

In [29]:
save_df_as_csv(df, "../data/all_projects.csv")
save_df_as_csv(geo_coords, "../data/project_coords.csv")